In [ ]:
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.compose import ColumnTransformer
#from sklearn.model_selection import train_test_split
#import xgboost as xgb
#from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
#import scipy.stats as stats
#from statsmodels.graphics.tsaplots import plot_acf
#from scipy.stats import skew, kurtosis, probplot
#from statsmodels.stats.stattools import jarque_bera
#from statsmodels.stats.diagnostic import acorr_ljungbox
#from scipy.stats import shapiro
#from sklearn.model_selection import TimeSeriesSplit, cross_val_score
#from lightgbm import LGBMRegressor
#import lightgbm as lgb
#from lightgbm import early_stopping
#from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error, r2_score, make_scorer
#from sklearn.model_selection import cross_validate
#from sklearn.model_selection import GridSearchCV
#import plotly.graph_objects as go
#import joblib

In [ ]:
#import warnings
#warnings.simplefilter(action='ignore', category=Warning)

In [ ]:
#!pip install mlflow
#!pip install --upgrade jinja2
#!pip install --upgrade Flask
#!pip install setuptools

### Plan d’Action
Démarrer le serveur MLflow (tu l’as déjà fait)
Créer l’expérience MLflow adaptée à ton projet
Enregistrer les modèles et métriques XGBoost et LGBM dans MLflow
Tester que tout fonctionne

### Démarrer MLflow

In [21]:

# starts an MLflow server locally.
!mlflow server --host 127.0.0.1 --port 8080

^C


### Configurer MLflow pour notre projet

In [22]:
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# Connexion au serveur MLflow
mlflow.set_tracking_uri("http://127.0.0.1:8080")

# Créer une expérience MLflow spécifique à ton projet
experiment_name = "Forecasting_Energie"
mlflow.set_experiment(experiment_name)

client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = client.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

print(f"Expérience MLflow créée: {experiment_name} (ID: {experiment_id})")


2025/03/20 22:46:41 INFO mlflow.tracking.fluent: Experiment with name 'Forecasting_Energie' does not exist. Creating a new experiment.


Expérience MLflow créée: Forecasting_Energie (ID: 494854308983366684)


In [23]:
df = pd.read_csv("C:/Projet_MLOPS/data/df_final_ML2.csv")


### Conversions 

In [24]:
#Convertir la colonne Date en datetime
df["Date"] = pd.to_datetime(df["Date"])

#Convertir la colonne Code INSEE région en object
df['Code INSEE région'] = df['Code INSEE région'].astype('object')

#Convertir la colonne Loi energie climat 2019 en integer

df['Loi energie climat 2019'] = df['Loi energie climat 2019'].astype('int')

### Feature engineering 

En plus des variables temporelles ajoutés dans la partie data management (jour, mois, année, sinus, cosinus, etc), nous allons également procéder à la construction de lags

In [25]:
# Ajout des lags pour la consommation d'énergie jusqu'à 90 jours
for lag in [1, 7, 30, 60, 90, 365]:  # Lags de 1, 7, 30, 60 et 90 jours
    df[f'lag_{lag}_Consommation'] = df['Consommation (MWh)'].shift(lag)

# Ajout des lags pour les variables climatiques jusqu'à 90 jours
for lag in [1, 7, 30, 60, 90]:
    df[f'lag_{lag}_TMin'] = df['TMin (°C)'].shift(lag)
    df[f'lag_{lag}_TMax'] = df['TMax (°C)'].shift(lag)
    df[f'lag_{lag}_TMoy'] = df['TMoy (°C)'].shift(lag)
    df[f'lag_{lag}_Vitesse_vent'] = df['Vitesse du vent à 100m (m/s)'].shift(lag)
    df[f'lag_{lag}_Rayonnement'] = df['Rayonnement solaire global (W/m2)'].shift(lag)

# Supprimer les valeurs manquantes créées par les lags
df = df.dropna()

In [26]:
df = df.sort_values(['Code INSEE région', 'Date'])

# Définir les fenêtres de rolling
windows = [7, 30, 90]  # 7 jours, 30 jours, 90 jours

for window in windows:
    df[f'Consommation_rolling_mean_{window}'] = df.groupby('Code INSEE région')['Consommation (MWh)'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())

#Gestion des NA : 

# Remplir les NaN avec la méthode de forward fill **par région**
df['Consommation_rolling_mean_7'] = df.groupby('Code INSEE région')['Consommation_rolling_mean_7'].fillna(method='ffill')
df['Consommation_rolling_mean_30'] = df.groupby('Code INSEE région')['Consommation_rolling_mean_30'].fillna(method='ffill')
df['Consommation_rolling_mean_90'] = df.groupby('Code INSEE région')['Consommation_rolling_mean_90'].fillna(method='ffill')


C:\Users\hp\AppData\Local\Temp\ipykernel_29604\13998302.py:12: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df['Consommation_rolling_mean_7'] = df.groupby('Code INSEE région')['Consommation_rolling_mean_7'].fillna(method='ffill')
C:\Users\hp\AppData\Local\Temp\ipykernel_29604\13998302.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Consommation_rolling_mean_7'] = df.groupby('Code INSEE région')['Consommation_rolling_mean_7'].fillna(method='ffill')
C:\Users\hp\AppData\Local\Temp\ipykernel_29604\13998302.py:13: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single val

### Gestion des outliers

In [27]:
# Calculer Q1, Q3 et l'IQR pour la target
Q1 = df['Consommation (MWh)'].quantile(0.25)
Q3 = df['Consommation (MWh)'].quantile(0.75)
IQR = Q3 - Q1

# Identifier les outliers dans la consommation
outliers = (df['Consommation (MWh)'] < (Q1 - 1.5 * IQR)) | (df['Consommation (MWh)'] > (Q3 + 1.5 * IQR))

# Supprimer les lignes avec outliers dans la target
df_cleaned = df[~outliers]

### Train / test split et transformations

In [28]:
df_transformed = df_cleaned.copy() #copier le data set pour ne pas écraser les modifications

In [30]:
import mlflow
import mlflow.sklearn
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Séparer la target (y) des features (X)
df_transformed = df_transformed.sort_values(by=['Date', 'Code INSEE région']).\
  reset_index(drop=True)

X = df_transformed[['Région','Thermique (MWh)', 
       'Hydraulique (MWh)', 'Bioénergies (MWh)', 
       'TMoy (°C)', 'Semaine_cos','Semaine_sin', 'lag_7_Consommation', 'lag_30_Consommation', 'lag_365_Consommation', 
                    'lag_7_TMoy', 'Consommation_rolling_mean_7']]

y = df_transformed['Consommation (MWh)']

# Split train/test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)#shuffle = False pour conserver l'ordre chronologique

In [32]:
from sklearn.preprocessing import OneHotEncoder
#Encodage de la variable Région
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit uniquement sur X_train et transformer X_train
X_train_encoded = encoder.fit_transform(X_train[['Région']])
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(['Région']), index=X_train.index)

# Transformer X_test (sans refit)
X_test_encoded = encoder.transform(X_test[['Région']])
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(['Région']), index=X_test.index)

# Concaténer les colonnes encodées avec les autres features
X_train = pd.concat([X_train.drop(columns=['Région']), X_train_encoded_df], axis=1)
X_test = pd.concat([X_test.drop(columns=['Région']), X_test_encoded_df], axis=1)

In [33]:
# Concaténer X_train et y_train pour faciliter l'analyse
df_train = X_train[['Région_Auvergne-Rhône-Alpes',
       'Région_Bourgogne-Franche-Comté', 'Région_Bretagne',
       'Région_Centre-Val de Loire', 'Région_Grand Est',
       'Région_Hauts-de-France', 'Région_Normandie',
       'Région_Nouvelle-Aquitaine', 'Région_Occitanie',
       'Région_Pays de la Loire', "Région_Provence-Alpes-Côte d'Azur",
       'Région_Île-de-France']].copy()
df_train['Consommation (MWh)'] = y_train  

# Calculer la corrélation de chaque feature avec la target
correlations = df_train.corr()['Consommation (MWh)'].drop('Consommation (MWh)')  

# Trier par ordre décroissant d'importance absolue
correlations = correlations.abs().sort_values(ascending=False)

# Afficher les variables les plus corrélées
print(correlations)


Région_Auvergne-Rhône-Alpes          0.452152
Région_Île-de-France                 0.438790
Région_Centre-Val de Loire           0.353461
Région_Bourgogne-Franche-Comté       0.305078
Région_Bretagne                      0.284825
Région_Hauts-de-France               0.215121
Région_Pays de la Loire              0.198573
Région_Normandie                     0.188714
Région_Grand Est                     0.132741
Région_Nouvelle-Aquitaine            0.098058
Région_Provence-Alpes-Côte d'Azur    0.045452
Région_Occitanie                     0.004412
Name: Consommation (MWh), dtype: float64


In [34]:
#Garder uniquement les régions avec une corrélation supérieur à 0.30
X_train = X_train.drop(columns=['Région_Bretagne',
        'Région_Grand Est','Région_Hauts-de-France', 'Région_Normandie',
       'Région_Nouvelle-Aquitaine', 'Région_Occitanie',
       'Région_Pays de la Loire', "Région_Provence-Alpes-Côte d'Azur"
       ])

X_test = X_test.drop(columns=['Région_Bretagne',
        'Région_Grand Est','Région_Hauts-de-France', 'Région_Normandie',
       'Région_Nouvelle-Aquitaine', 'Région_Occitanie',
       'Région_Pays de la Loire', "Région_Provence-Alpes-Côte d'Azur"
       ])

In [36]:
from sklearn.preprocessing import StandardScaler
# Colonnes numériques à standardiser
normalize_cols = ['Thermique (MWh)', 
       'Hydraulique (MWh)', 'Bioénergies (MWh)', 
       'TMoy (°C)',  'lag_7_Consommation', 'lag_30_Consommation', 'lag_365_Consommation', 
                    'lag_7_TMoy', 'Consommation_rolling_mean_7']

# Initialisation des scalers
scaler_X = StandardScaler()

# Standardiser X_train 
X_train[normalize_cols] = scaler_X.fit_transform(X_train[normalize_cols])

# Appliquer la transformation sur  X_test (sans refit)
X_test[normalize_cols] = scaler_X.transform(X_test[normalize_cols])


In [37]:
X_train.to_csv("X_train_vf2.csv", index=False)

### Entraînement du premier modèle ML XGBoost

Utilisation de grid search pour nous aider dans le choix des hyperparamètres

In [51]:
import mlflow
import mlflow.sklearn
import xgboost as xgb
import numpy as np
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# Définir le modèle XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Définir les hyperparamètres à tester avec GridSearchCV
param_grid = {
    'n_estimators': [400, 500],  # Nombre d'estimateurs
    'learning_rate': [0.01, 0.05, 0.1],  # Taux d'apprentissage
    'max_depth': [5, 6, 7],  # Profondeur des arbres
    'colsample_bytree': [0.7, 0.8, 1.0],  # Proportion de features pour chaque arbre
    'gamma': [0, 0.3],  # Lutte contre l’overfitting en contrôlant les branches des arbres
}

# GridSearchCV : recherche exhaustive des meilleurs paramètres
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=3, scoring='neg_mean_absolute_error', 
                           n_jobs=-1, verbose=1)

# Démarrer un suivi d'expérience avec MLflow
with mlflow.start_run():

    # Entraîner le modèle avec la recherche sur la grille
    grid_search.fit(X_train, y_train)

    # Afficher les meilleurs paramètres trouvés
    print(f"Meilleurs paramètres : {grid_search.best_params_}")
    
    # Log des meilleurs paramètres
    mlflow.log_params(grid_search.best_params_)

    # Utiliser le modèle optimal pour faire des prédictions
    best_model_xgb = grid_search.best_estimator_

    # Validation croisée avec TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=3)
    
    mae_scores, mse_scores, mape_scores, r2_scores = [], [], [], []

    for train_index, test_index in tscv.split(X_train):
        X_train_iter, X_test_iter = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_iter, y_test_iter = y_train.iloc[train_index], y_train.iloc[test_index]

        # Entraînement avec le meilleur modèle trouvé par GridSearchCV
        best_model_xgb.fit(X_train_iter, y_train_iter)

        # Prédictions
        y_pred = best_model_xgb.predict(X_test_iter)

        # Calcul des métriques
        mae = mean_absolute_error(y_test_iter, y_pred)
        mse = mean_squared_error(y_test_iter, y_pred)
        mape = np.mean(np.abs((y_test_iter - y_pred) / y_test_iter)) * 100
        r2 = r2_score(y_test_iter, y_pred)

        mae_scores.append(mae)
        mse_scores.append(mse)
        mape_scores.append(mape)
        r2_scores.append(r2)

    # Log des résultats de validation croisée dans MLflow
    mlflow.log_metric("MAE moyen", np.mean(mae_scores))
    mlflow.log_metric("MAPE moyen", np.mean(mape_scores))
    mlflow.log_metric("MSE moyen", np.mean(mse_scores))
    mlflow.log_metric("R² moyen", np.mean(r2_scores))

    # Prédictions sur l'ensemble de test et de train
    y_pred_train = best_model_xgb.predict(X_train)
    y_pred_test = best_model_xgb.predict(X_test)

    # Calcul des métriques sur l'ensemble de train
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    mape_train = np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100  # MAPE en %

    # Calcul des métriques sur l'ensemble de test
    mae_test = mean_absolute_error(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    mape_test = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100  # MAPE en %

    # Log des résultats de test et train dans MLflow
    mlflow.log_metric("MAE train", mae_train)
    mlflow.log_metric("MSE train", mse_train)
    mlflow.log_metric("R² train", r2_train)
    mlflow.log_metric("MAPE train", mape_train)

    mlflow.log_metric("MAE test", mae_test)
    mlflow.log_metric("MSE test", mse_test)
    mlflow.log_metric("R² test", r2_test)
    mlflow.log_metric("MAPE test", mape_test)

    # Sauvegarder le modèle 'best_model_xgb' dans un fichier
    joblib.dump(best_model_xgb, 'best_model_xgb.pkl')

    # Log du modèle XGBoost dans MLflow
    mlflow.sklearn.log_model(best_model_xgb, "best_model")

    # Charger le modèle sauvegardé
    best_model_xgb_final = joblib.load('best_model_xgb.pkl')

    print(f"MAE train : {mae_train}")
    print(f"MSE train: {mse_train}")
    print(f"R² train : {r2_train}")
    print(f"MAPE train : {mape_train}%")

    print(f"MAE test : {mae_test}")
    print(f"MSE test : {mse_test}")
    print(f"R² test : {r2_test}")
    print(f"MAPE test : {mape_test}%")


Fitting 3 folds for each of 108 candidates, totalling 324 fits
🏃 View run kindly-owl-479 at: http://127.0.0.1:8080/#/experiments/494854308983366684/runs/f184e90ca79642ddadbe5aec5864c868
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/494854308983366684


KeyboardInterrupt: 

### Modèle LGBM

In [ ]:
# Sauvegarder le modèle 'best_model_lgb' dans un fichier
joblib.dump(best_model_lgb, 'best_model_lgbm.pkl')

In [ ]:
# Charger le modèle sauvegardé
best_model_loaded= joblib.load('best_model_lgbm.pkl')

#### LightGBM

In [ ]:
import mlflow
import mlflow.sklearn
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis, jarque_bera, shapiro
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Initialiser MLflow
mlflow.set_experiment("LightGBM_Experiment")

with mlflow.start_run():
    # Définition du modèle LightGBM
    model_lgb = lgb.LGBMRegressor(random_state=42)

    # Hyperparamètres à tester avec GridSearchCV
    param_grid_lgb = {
        'n_estimators': [200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'num_leaves': [31, 40, 50],
        'colsample_bytree': [0.7, 0.8, 1.0],
        'subsample': [0.8, 1.0]
    }

    # GridSearchCV pour la recherche des meilleurs hyperparamètres
    grid_search_lgb = GridSearchCV(estimator=model_lgb, param_grid=param_grid_lgb, 
                                   cv=3, scoring='neg_mean_absolute_error', 
                                   n_jobs=-1, verbose=1)

    # Entraîner le modèle avec GridSearch
    grid_search_lgb.fit(X_train, y_train)

    # Récupérer le meilleur modèle et hyperparamètres
    best_model_lgb = grid_search_lgb.best_estimator_
    best_params = grid_search_lgb.best_params_

    # Log des hyperparamètres trouvés par GridSearch
    mlflow.log_params(best_params)

    # Validation croisée avec TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=3)
    mae_scores, mse_scores, mape_scores, r2_scores = [], [], [], []

    for train_index, test_index in tscv.split(X_train):
        X_train_iter, X_test_iter = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_iter, y_test_iter = y_train.iloc[train_index], y_train.iloc[test_index]

        # Entraîner le meilleur modèle
        best_model_lgb.fit(X_train_iter, y_train_iter)

        # Prédictions
        y_pred = best_model_lgb.predict(X_test_iter)

        # Calcul des métriques
        mae = mean_absolute_error(y_test_iter, y_pred)
        mse = mean_squared_error(y_test_iter, y_pred)
        mape = np.mean(np.abs((y_test_iter - y_pred) / y_test_iter)) * 100
        r2 = r2_score(y_test_iter, y_pred)

        mae_scores.append(mae)
        mse_scores.append(mse)
        mape_scores.append(mape)
        r2_scores.append(r2)

    # Moyennes des métriques
    avg_mae, avg_mse, avg_mape, avg_r2 = np.mean(mae_scores), np.mean(mse_scores), np.mean(mape_scores), np.mean(r2_scores)

    # Log des métriques dans MLflow
    mlflow.log_metric("MAE_CV", avg_mae)
    mlflow.log_metric("MSE_CV", avg_mse)
    mlflow.log_metric("MAPE_CV", avg_mape)
    mlflow.log_metric("R2_CV", avg_r2)

    # Sauvegarde du meilleur modèle
    joblib.dump(best_model_lgb, 'best_model_lgbm.pkl')
    mlflow.sklearn.log_model(best_model_lgb, "model")

    # Prédictions sur l'ensemble train et test
    y_pred_train = best_model_lgb.predict(X_train)
    y_pred_test = best_model_lgb.predict(X_test)

    # Calcul des métriques finales
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    mape_train = np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100

    mae_test = mean_absolute_error(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    mape_test = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

    # Log des métriques finales
    mlflow.log_metric("MAE_Train", mae_train)
    mlflow.log_metric("MSE_Train", mse_train)
    mlflow.log_metric("MAPE_Train", mape_train)
    mlflow.log_metric("R2_Train", r2_train)

    mlflow.log_metric("MAE_Test", mae_test)
    mlflow.log_metric("MSE_Test", mse_test)
    mlflow.log_metric("MAPE_Test", mape_test)
    mlflow.log_metric("R2_Test", r2_test)

    print(f"MAE Train: {mae_train}, MAE Test: {mae_test}")
    print(f"MSE Train: {mse_train}, MSE Test: {mse_test}")
    print(f"R² Train: {r2_train}, R² Test: {r2_test}")
    print(f"MAPE Train: {mape_train}%, MAPE Test: {mape_test}%")

    # Analyse des résidus
    residuals = y_test - y_pred_test

    plt.hist(residuals, bins=50, alpha=0.75)
    plt.title("Distribution des résidus")
    plt.xlabel("Résidus")
    plt.ylabel("Fréquence")
    plt.savefig("residuals_distribution.png")
    mlflow.log_artifact("residuals_distribution.png")

    # Autres statistiques sur les résidus
    skewness_residus = skew(residuals)
    kurtosis_residus = kurtosis(residuals)

    mlflow.log_metric("Skewness_Residus", skewness_residus)
    mlflow.log_metric("Kurtosis_Residus", kurtosis_residus)

    print(f"Skewness des résidus : {skewness_residus}")
    print(f"Kurtosis des résidus : {kurtosis_residus}")

    # Test de normalité des résidus
    jb_test = jarque_bera(residuals)
    sw_test = shapiro(residuals)
    
    mlflow.log_metric("Jarque-Bera", jb_test[0])
    mlflow.log_metric("Shapiro-Wilk", sw_test[0])

    print("Test Jarque-Bera:", jb_test)
    print("Test Shapiro-Wilk:", sw_test)

    # Fin de l'expérience MLflow
    mlflow.end_run()
